In [1]:
# lstmで日経平均の上下の予測(多変量)2

In [2]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/kaggle/RNN

Mounted at /content/drive
/content/drive/MyDrive/kaggle/RNN


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
# データの読み込み
train1 = pd.read_csv("data/N225_2003.csv")
train2 = pd.read_csv("data/NYダウ平均株価2003.csv")
train3 = pd.read_csv("data/USD_JPY2003.csv")
print(train1.head())
print(train2.head())
print(train3.head())

           日付け         終値         始値         高値         安値      出来高    前日比%
0  2021年12月30日  28,791.71  28,794.24  28,904.42  28,579.49  403.89M  -0.40%
1  2021年12月29日  28,906.88  28,995.73  29,106.28  28,729.61  447.11M  -0.56%
2  2021年12月28日  29,069.16  28,953.32  29,121.01  28,879.68  470.40M   1.37%
3  2021年12月27日  28,676.46  28,786.33  28,805.28  28,658.82  374.87M  -0.37%
4  2021年12月24日  28,782.59  28,836.05  28,870.13  28,773.50  359.09M  -0.05%
           日付け         終値         始値         高値         安値      出来高    前日比%
0  2021年12月31日  36,338.30  36,385.85  36,484.92  36,304.53  205.29M  -0.16%
1  2021年12月30日  36,398.08  36,558.11  36,679.09  36,372.13  197.77M  -0.25%
2  2021年12月29日  36,488.63  36,421.14  36,569.85  36,398.51  206.45M   0.25%
3  2021年12月28日  36,398.67  36,322.51  36,526.89  36,318.73  231.82M   0.26%
4  2021年12月27日  36,302.58  36,023.79  36,304.17  35,985.01  241.55M   0.98%
           日付け      終値      始値      高値      安値    前日比%
0  2022年01月02日  115.14  115.12  1

In [5]:
# データフレームを結合
train = pd.merge(train1, train2, on='日付け', how='left')
train = pd.merge(train, train3, on='日付け', how='left')
train = train.dropna(how='any')
print(train.head(), train.isnull().sum())

           日付け       終値_x       始値_x       高値_x  ...      始値      高値      安値    前日比%
0  2021年12月30日  28,791.71  28,794.24  28,904.42  ...  114.96  115.21  114.93   0.10%
1  2021年12月29日  28,906.88  28,995.73  29,106.28  ...  114.82  115.04  114.67   0.11%
2  2021年12月28日  29,069.16  28,953.32  29,121.01  ...  114.87  114.96  114.71  -0.02%
3  2021年12月27日  28,676.46  28,786.33  28,805.28  ...  114.39  114.93  114.30   0.37%
5  2021年12月23日  28,798.37  28,703.01  28,798.37  ...  114.10  114.48  114.07   0.23%

[5 rows x 18 columns] 日付け       0
終値_x      0
始値_x      0
高値_x      0
安値_x      0
出来高_x     0
前日比%_x    0
終値_y      0
始値_y      0
高値_y      0
安値_y      0
出来高_y     0
前日比%_y    0
終値        0
始値        0
高値        0
安値        0
前日比%      0
dtype: int64


In [6]:
train = train.iloc[::-1].reset_index(drop=True)
train.head()

,日付け,終値_x,始値_x,高値_x,安値_x,出来高_x,前日比%_x,終値_y,始値_y,高値_y,安値_y,出来高_y,前日比%_y,終値,始値,高値,安値,前日比%
0,2003年01月06日,"8,713.33","8,669.89","8,761.78","8,669.89",-,1.57%,"8,773.57","8,602.78","8,800.59","8,602.10",258.04M,2.00%,119.18,119.80,119.91,118.52,-0.46%
1,2003年01月07日,"8,656.50","8,810.74","8,829.06","8,654.52",-,-0.65%,"8,740.59","8,775.84","8,802.64","8,713.03",266.97M,-0.38%,120.43,119.19,120.64,119.05,1.05%
2,2003年01月08日,"8,517.80","8,614.41","8,614.41","8,491.49",-,-1.60%,"8,595.31","8,735.93","8,736.07","8,580.10",255.65M,-1.66%,119.06,120.46,120.55,118.62,-1.14%
3,2003年01月09日,"8,497.93","8,440.32","8,498.03","8,400.59",-,-0.23%,"8,776.18","8,596.68","8,787.70","8,596.61",253.56M,2.10%,119.32,119.10,119.64,118.67,0.22%
4,2003年01月10日,"8,470.45","8,563.38","8,569.85","8,378.18",-,-0.32%,"8,784.89","8,776.04","8,818.55","8,689.58",235.61M,0.10%,119.28,119.27,120.08,119.04,-0.03%


In [7]:
# データに含まれるカンマを取り除いて、型をfloatに変換
train['前日比%_x'] = train['前日比%_x'].str.replace('%', '')
train['終値_x'] = train['終値_x'].str.replace(',', '')
train['始値_x'] = train['始値_x'].str.replace(',', '')
train['高値_x'] = train['高値_x'].str.replace(',', '')
train['安値_x'] = train['安値_x'].str.replace(',', '')
train['前日比%_y'] = train['前日比%_y'].str.replace('%', '')
train['終値_y'] = train['終値_y'].str.replace(',', '')
train['始値_y'] = train['始値_y'].str.replace(',', '')
train['高値_y'] = train['高値_y'].str.replace(',', '')
train['安値_y'] = train['安値_y'].str.replace(',', '')
train['前日比%'] = train['前日比%'].str.replace('%', '')
train = train.drop(['日付け', '出来高_x',  '出来高_y'] , axis = 1)
print(train)
train = train.astype(float)
train.info()

          終値_x      始値_x      高値_x      安値_x  ...      始値      高値      安値   前日比%
0      8713.33   8669.89   8761.78   8669.89  ...  119.80  119.91  118.52  -0.46
1      8656.50   8810.74   8829.06   8654.52  ...  119.19  120.64  119.05   1.05
2      8517.80   8614.41   8614.41   8491.49  ...  120.46  120.55  118.62  -1.14
3      8497.93   8440.32   8498.03   8400.59  ...  119.10  119.64  118.67   0.22
4      8470.45   8563.38   8569.85   8378.18  ...  119.27  120.08  119.04  -0.03
...        ...       ...       ...       ...  ...     ...     ...     ...    ...
4495  28798.37  28703.01  28798.37  28640.15  ...  114.10  114.48  114.07   0.23
4496  28676.46  28786.33  28805.28  28658.82  ...  114.39  114.93  114.30   0.37
4497  29069.16  28953.32  29121.01  28879.68  ...  114.87  114.96  114.71  -0.02
4498  28906.88  28995.73  29106.28  28729.61  ...  114.82  115.04  114.67   0.11
4499  28791.71  28794.24  28904.42  28579.49  ...  114.96  115.21  114.93   0.10

[4500 rows x 15 columns]
<c

In [8]:
for s in range(len(train)):
  if s > 0:
    train['前日比%_x'][s] = train['終値_x'][s] - train['終値_x'][s-1]
    train['前日比%_y'][s] = train['終値_y'][s] - train['終値_y'][s-1]
    train['前日比%'][s] = train['終値'][s] - train['終値'][s-1]
  else:
    train['前日比%_x'][0] = 0
    train['前日比%_y'][0] = 0
    train['前日比%'][0] = 0


In [9]:
train

,終値_x,始値_x,高値_x,安値_x,前日比%_x,終値_y,始値_y,高値_y,安値_y,前日比%_y,終値,始値,高値,安値,前日比%
0,8713.33,8669.89,8761.78,8669.89,0.00,8773.57,8602.78,8800.59,8602.10,0.00,119.18,119.80,119.91,118.52,0.00
1,8656.50,8810.74,8829.06,8654.52,-56.83,8740.59,8775.84,8802.64,8713.03,-32.98,120.43,119.19,120.64,119.05,1.25
2,8517.80,8614.41,8614.41,8491.49,-138.70,8595.31,8735.93,8736.07,8580.10,-145.28,119.06,120.46,120.55,118.62,-1.37
3,8497.93,8440.32,8498.03,8400.59,-19.87,8776.18,8596.68,8787.70,8596.61,180.87,119.32,119.10,119.64,118.67,0.26
4,8470.45,8563.38,8569.85,8378.18,-27.48,8784.89,8776.04,8818.55,8689.58,8.71,119.28,119.27,120.08,119.04,-0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,28798.37,28703.01,28798.37,28640.15,236.16,35950.63,35846.37,36060.41,35833.84,196.67,114.37,114.10,114.48,114.07,0.26
4496,28676.46,28786.33,28805.28,28658.82,-121.91,36302.58,36023.79,36304.17,35985.01,351.95,114.83,114.39,114.93,114.30,0.46
4497,29069.16,28953.32,29121.01,28879.68,392.70,36398.67,36322.51,36526.89,36318.73,96.09,114.81,114.87,114.96,114.71,-0.02
4498,28906.88,28995.73,29106.28,28729.61,-162.28,36488.63,36421.14,36569.85,36398.51,89.96,114.94,114.82,115.04,114.67,0.13


In [10]:
# 標準化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train)
train_scaled = scaler.transform(train)
print('train_scaled = \n', train_scaled[0:5], len(train_scaled))

train_scaled = 
 [[-1.22446915e+00 -1.23248944e+00 -1.22798930e+00 -1.21986757e+00
  -2.04253546e-02 -1.07280271e+00 -1.09651813e+00 -1.07718780e+00
  -1.08775955e+00 -3.06258371e-02  1.16036986e+00  1.21218422e+00
   1.18108608e+00  1.14903164e+00  1.38946323e-03]
 [-1.23453384e+00 -1.20754734e+00 -1.21611711e+00 -1.22260043e+00
  -2.80579713e-01 -1.07744679e+00 -1.07214543e+00 -1.07690026e+00
  -1.07207274e+00 -1.95160176e-01  1.26244970e+00  1.16235400e+00
   1.24057888e+00  1.19240657e+00  1.89841147e+00]
 [-1.25909785e+00 -1.24231399e+00 -1.25399410e+00 -1.25158796e+00
  -6.55361296e-01 -1.09790437e+00 -1.07776611e+00 -1.08623773e+00
  -1.09087061e+00 -7.55415064e-01  1.15057020e+00  1.26609888e+00
   1.23324415e+00  1.15721559e+00 -2.07774665e+00]
 [-1.26261687e+00 -1.27314231e+00 -1.27453044e+00 -1.26775042e+00
  -1.11385536e-01 -1.07243518e+00 -1.09737722e+00 -1.07899583e+00
  -1.08853590e+00  8.71718790e-01  1.17180280e+00  1.15500200e+00
   1.15908189e+00  1.16130757e+00  3.9

In [11]:
# 日経平均の上下
y_data = []
for k in range(len(train)):
  if train['前日比%_x'][k] < 0:
    #train['前日比%_x'][k] = 0
    y_data.append(0)
  else:
    #train['前日比%_x'][k] = 1
    y_data.append(1)

print(y_data[0:10])
print(len(y_data))

[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]
4500


In [12]:
# 訓練データとテストデータに分割

x_train = train_scaled[0:4200]
x_test = train_scaled[4200:4450]
print('x_train = \n', x_train[0:5], len(x_train))
print('x_test = \n', x_test[0:5], len(x_test))

y_train = y_data[51:4201]
y_test = y_data[4251:4451]
print('y_train = \n', y_train[0:5], len(y_train))
print('y_test = \n', y_test[0:5], len(y_test))

x_train = 
 [[-1.22446915e+00 -1.23248944e+00 -1.22798930e+00 -1.21986757e+00
  -2.04253546e-02 -1.07280271e+00 -1.09651813e+00 -1.07718780e+00
  -1.08775955e+00 -3.06258371e-02  1.16036986e+00  1.21218422e+00
   1.18108608e+00  1.14903164e+00  1.38946323e-03]
 [-1.23453384e+00 -1.20754734e+00 -1.21611711e+00 -1.22260043e+00
  -2.80579713e-01 -1.07744679e+00 -1.07214543e+00 -1.07690026e+00
  -1.07207274e+00 -1.95160176e-01  1.26244970e+00  1.16235400e+00
   1.24057888e+00  1.19240657e+00  1.89841147e+00]
 [-1.25909785e+00 -1.24231399e+00 -1.25399410e+00 -1.25158796e+00
  -6.55361296e-01 -1.09790437e+00 -1.07776611e+00 -1.08623773e+00
  -1.09087061e+00 -7.55415064e-01  1.15057020e+00  1.26609888e+00
   1.23324415e+00  1.15721559e+00 -2.07774665e+00]
 [-1.26261687e+00 -1.27314231e+00 -1.27453044e+00 -1.26775042e+00
  -1.11385536e-01 -1.07243518e+00 -1.09737722e+00 -1.07899583e+00
  -1.08853590e+00  8.71718790e-01  1.17180280e+00  1.15500200e+00
   1.15908189e+00  1.16130757e+00  3.959700

In [13]:
# ５０行１セットで訓練データを作成
length = 50
x_train_data = []
x_test_data = []

y_data = [] #予測したいデータ

for i in range(len(x_train)-length):
    x_train_data.append(x_train[i:i+50])

for i in range(len(x_test)-length):
    x_test_data.append(x_test[i:i+50])

x_train = np.asarray(x_train_data)
x_test = np.asarray(x_test_data)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

print(x_train.shape)
print(x_test.shape)

print(len(y_train))
print(len(y_test))

(4150, 50, 15)
(200, 50, 15)
4150
200


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation  

# 多変量lstmで予測
data_dim = 15
length_of_sequences = 50
h_num = 100
in_out_neurons = 1

model = Sequential()
model.add(LSTM(h_num, input_shape=(50, 15), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dense(20))
model.add(Dense(in_out_neurons))
model.add(Activation("sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"
model.fit(x_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 6s 17ms/step - loss: 0.2565
Epoch 2/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2508
Epoch 3/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2510
Epoch 4/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2510
Epoch 5/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2498
Epoch 6/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2497
Epoch 7/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2494
Epoch 8/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2493
Epoch 9/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2490
Epoch 10/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2491
Epoch 11/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2492
Epoch 12/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2492
Epoch 13/100
42/42 [=====

In [15]:
y_submit = []
y_submit = model.predict(x_test, batch_size=100, verbose=1)
print(y_submit.shape)
print(y_submit[0:5])
for k in range(len(y_submit)):
  if y_submit[k] > 0.5:
    y_submit[k] = 1
  else:
    y_submit[k] = 0
accuracy = 0
for i in range(len(y_submit)):
  if y_submit[i] == y_test[i]:
    accuracy += 1
accuracy = accuracy/len(y_submit)*100
print('accuracy =', accuracy, '%')

2/2 [==============================] - 1s 12ms/step
(200, 1)
[[0.25225884]
 [0.17011452]
 [0.76436836]
 [0.07554156]
 [0.9703636 ]]
accuracy = 47.0 %


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation  

# 多変量lstmで予測
data_dim = 15
length_of_sequences = 50
h_num = 100
in_out_neurons = 1

model = Sequential()
model.add(LSTM(h_num, input_shape=(50, 15), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dense(20))
model.add(Dense(in_out_neurons))
model.add(Activation("sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"
model.fit(x_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 3s 17ms/step - loss: 0.2549
Epoch 2/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2501
Epoch 3/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2497
Epoch 4/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2496
Epoch 5/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2496
Epoch 6/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2493
Epoch 7/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2493
Epoch 8/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2500
Epoch 9/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2493
Epoch 10/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2491
Epoch 11/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2504
Epoch 12/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2489
Epoch 13/100
42/42 [=====

In [17]:
y_submit = []
y_submit = model.predict(x_test, batch_size=100, verbose=1)
print(y_submit.shape)
print(y_submit[0:5])
for k in range(len(y_submit)):
  if y_submit[k] > 0.5:
    y_submit[k] = 1
  else:
    y_submit[k] = 0
accuracy = 0
for i in range(len(y_submit)):
  if y_submit[i] == y_test[i]:
    accuracy += 1
accuracy = accuracy/len(y_submit)*100
print('accuracy =', accuracy, '%')

2/2 [==============================] - 0s 8ms/step
(200, 1)
[[0.88418984]
 [0.41897732]
 [0.9355528 ]
 [0.06489094]
 [0.7616532 ]]
accuracy = 52.0 %


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation  

# 多変量lstmで予測
data_dim = 15
length_of_sequences = 50
h_num = 100
in_out_neurons = 1

model = Sequential()
model.add(LSTM(h_num, input_shape=(50, 15), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dense(20))
model.add(Dense(in_out_neurons))
model.add(Activation("sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"
model.fit(x_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 3s 16ms/step - loss: 0.2530
Epoch 2/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2512
Epoch 3/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2506
Epoch 4/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2501
Epoch 5/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2495
Epoch 6/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2499
Epoch 7/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2496
Epoch 8/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2497
Epoch 9/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2487
Epoch 10/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2494
Epoch 11/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2495
Epoch 12/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2491
Epoch 13/100
42/42 [=====

In [19]:
y_submit = []
y_submit = model.predict(x_test, batch_size=100, verbose=1)
print(y_submit.shape)
print(y_submit[0:5])
for k in range(len(y_submit)):
  if y_submit[k] > 0.5:
    y_submit[k] = 1
  else:
    y_submit[k] = 0
accuracy = 0
for i in range(len(y_submit)):
  if y_submit[i] == y_test[i]:
    accuracy += 1
accuracy = accuracy/len(y_submit)*100
print('accuracy =', accuracy, '%')

2/2 [==============================] - 0s 9ms/step
(200, 1)
[[0.23917638]
 [0.11258014]
 [0.88470256]
 [0.0055558 ]
 [0.6482641 ]]
accuracy = 50.0 %


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation  

# 多変量lstmで予測
data_dim = 15
length_of_sequences = 50
h_num = 100
in_out_neurons = 1

model = Sequential()
model.add(LSTM(h_num, input_shape=(50, 15), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dense(20))
model.add(Dense(in_out_neurons))
model.add(Activation("sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"
model.fit(x_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 3s 16ms/step - loss: 0.2524
Epoch 2/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2508
Epoch 3/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2497
Epoch 4/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2499
Epoch 5/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2502
Epoch 6/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2498
Epoch 7/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2495
Epoch 8/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2492
Epoch 9/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2495
Epoch 10/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2497
Epoch 11/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2493
Epoch 12/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2491
Epoch 13/100
42/42 [=====

In [21]:
y_submit = []
y_submit = model.predict(x_test, batch_size=100, verbose=1)
print(y_submit.shape)
print(y_submit[0:5])
for k in range(len(y_submit)):
  if y_submit[k] > 0.5:
    y_submit[k] = 1
  else:
    y_submit[k] = 0
accuracy = 0
for i in range(len(y_submit)):
  if y_submit[i] == y_test[i]:
    accuracy += 1
accuracy = accuracy/len(y_submit)*100
print('accuracy =', accuracy, '%')

2/2 [==============================] - 0s 9ms/step
(200, 1)
[[0.95883566]
 [0.79676265]
 [0.56887615]
 [0.3127696 ]
 [0.08253913]]
accuracy = 51.0 %


In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation  

# 多変量lstmで予測
data_dim = 15
length_of_sequences = 50
h_num = 100
in_out_neurons = 1

model = Sequential()
model.add(LSTM(h_num, input_shape=(50, 15), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dense(20))
model.add(Dense(in_out_neurons))
model.add(Activation("sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"
model.fit(x_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 3s 16ms/step - loss: 0.2536
Epoch 2/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2516
Epoch 3/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2508
Epoch 4/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2509
Epoch 5/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2494
Epoch 6/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2506
Epoch 7/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2497
Epoch 8/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2499
Epoch 9/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2494
Epoch 10/100
42/42 [==============================] - 1s 16ms/step - loss: 0.2492
Epoch 11/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2489
Epoch 12/100
42/42 [==============================] - 1s 17ms/step - loss: 0.2492
Epoch 13/100
42/42 [=====

In [23]:
y_submit = []
y_submit = model.predict(x_test, batch_size=100, verbose=1)
print(y_submit.shape)
print(y_submit[0:5])
for k in range(len(y_submit)):
  if y_submit[k] > 0.5:
    y_submit[k] = 1
  else:
    y_submit[k] = 0
accuracy = 0
for i in range(len(y_submit)):
  if y_submit[i] == y_test[i]:
    accuracy += 1
accuracy = accuracy/len(y_submit)*100
print('accuracy =', accuracy, '%')

2/2 [==============================] - 0s 12ms/step
(200, 1)
[[0.01868984]
 [0.00565045]
 [0.9912696 ]
 [0.00176182]
 [0.99049765]]
accuracy = 56.99999999999999 %
